In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import datetime
from pprint import pprint as pp
import sys

sys.path.insert(0, '/home/molly/Desktop/DeepTCGA/')
import load_data

/home/molly/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def fc_layer(A_prev, size_in, size_out, name="fully-connected"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w) + b
        return act, w, b
    

def random_sparse_layer(A_prev, size_in, size_out, sparsity, name="sparse"):
    if sparsity < 0.01:
        mask = generate_really_sparse_mask(size_in, size_out, sparsity)
    else:
        mask = generate_random_sparse_mask(size_in, size_out, sparsity)
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        w_masked = tf.multiply(mask, w)
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w_masked) + b
        return act, w, w_masked, mask, b

    
def build_model(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2, b2 = fc_layer(hidden, N_HIDDEN, N_IN, name="fc1")
    parameters.update({"w2": w2, "b2": b2})
    return x_recon, parameters



def build_model_right_sparse(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1, b1 = fc_layer(x, N_IN, N_HIDDEN, name="fc1")
    parameters.update({"w1": w1, "b1": b1})    
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2_init, w2_sparse, mask2, b2 = random_sparse_layer(
        hidden, N_HIDDEN, N_IN, sparsity, name="sparse1")
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2})
    return x_recon, parameters


def build_model_both_sparse_assymetric(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2_init, w2_sparse, mask2, b2 = random_sparse_layer(
        hidden, N_HIDDEN, N_IN, sparsity, name="sparse2")
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2}) 
    return x_recon, parameters


def build_model_both_sparse_symetric(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    mask2 =tf.transpose(mask1)
    w2_init = tf.Variable(tf.truncated_normal([N_HIDDEN, N_IN], mean=0, stddev=0.1))
    w2_sparse = tf.multiply(mask2, w2_init)
    b2 = tf.Variable(tf.constant(0.1, shape=[N_IN]))
    x_recon = tf.matmul(hidden, w2_sparse) + b2
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2})     
    return x_recon, parameters


def back_prop(x, x_recon, learning_rate):
    loss = tf.reduce_mean(tf.square(x_recon - x))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_step


def feed_forward(x, parameters):
    w1_sparse, b1 = parameters["w1_sparse"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1_sparse) + b1)
    w2, b2 = parameters["w2"], parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def feed_forward_right_sparse(x, parameters):
    w1, b1 = parameters["w1"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2_sparse"] ,parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def feed_forward_both_sparse(x, parameters):
    w1, b1 = parameters["w1_sparse"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2_sparse"] ,parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden

def mse(x, x_recon, name=""):
    mse = tf.reduce_mean(tf.square(x_recon-x))
    mse_summary = tf.summary.scalar(name + "_mse", mse)
    return mse, mse_summary


def generate_random_sparse_mask(size_in, size_out, sparsity):
    """ make sure each node has at least one edge connecting to it"""
    assert(size_out>=100)
    while True:
        try:
            a = np.random.uniform(low=0, high=1, size=(size_in, size_out))
            a = (a>(1-sparsity)).astype(int)
            assert((a.sum(axis=1)>1).all())
            assert((a.sum(axis=0)>1).all())
            break
        except AssertionError:
            continue
    return tf.constant(a.astype('float32'))


def generate_really_sparse_mask(size_in, size_out, sparsity):
    a = np.random.uniform(low=0, high=1, size=(size_in, size_out))
    a = (a>(1-sparsity)).astype(int)
    return tf.constant(a.astype('float32'))

In [3]:
def train_model_random(data, batch_size=128, n_hidden=1000, 
                num_epoch=1000, learning_rate=1e-3, sparsity=1, mode="left", extra=""):
    tf.reset_default_graph()
    LOGDIR = "/tmp/tcga_{0}".format(datetime.datetime.today().date())
    N_IN = data.train.num_features
    N_OUT = data.train.num_features
    N_HIDDEN = n_hidden
    
    # train step
    (train_batch, train_iter, val_all, val_iter, 
        train_all, train_iter_all) = data.prep_batch(batch_size=batch_size, 
                                                     count_by="epoch")
    x = train_batch["X"]
    if mode=="left":
        x_recon, parameters = build_model(x, N_IN, N_HIDDEN, sparsity=sparsity)
    elif mode == "both":
        x_recon, parameters = build_model_both_sparse_symetric(x, N_IN, N_HIDDEN, sparsity=sparsity)        
    else:
        raise
    train_step = back_prop(x, x_recon, learning_rate)    
    
    # mse
    x_train, x_val = train_all["X"], val_all["X"]
    if mode=="left":
        x_train_recon, train_latent = feed_forward(x_train, parameters)
        x_val_recon, val_latent = feed_forward(x_val, parameters)
    elif mode == "both":
        x_train_recon, train_latent = feed_forward_both_sparse(x_train, parameters)
        x_val_recon, val_latent = feed_forward_both_sparse(x_val, parameters)
    train_mse, train_summ = mse(x_train, x_train_recon, name="train")
    val_mse, val_summ = mse(x_val, x_val_recon, name="valiation")
    
    # run
    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + "ae_{0}".format(extra))
    writer.add_graph(sess.graph)                            
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epoch):
        sess.run([train_iter.initializer])
        t0 = time.time()
        try:
            while True:
                sess.run(train_step)
        except tf.errors.OutOfRangeError:
            sess.run([train_iter_all.initializer, val_iter.initializer])
            [train_error, train_s, val_error, val_s] = sess.run(
                [train_mse, train_summ, val_mse, val_summ])
            writer.add_summary(train_s, epoch)
            writer.add_summary(val_s, epoch)
            if epoch % 10 == 0:
                print("epoch", epoch)
                print("training mse:", train_error, "validation mse", val_error)
                print("epoch time:", time.time()-t0)
#     train_latent = sess.run(train_latent)
#     val_latent = sess.run(val_latent)
#     np.save("./results/AE/train_latent_sparse2.npy", train_latent)
#     np.save("./results/AE/val_latent_sparse2.npy", val_latent)    
    sess.close()

In [4]:
tcga_pathway = load_data.read_data_sets(
    "../data/TF_network/mRNA_lognorm_StandardScaled_pathway_genes.hdf")

In [ ]:
for sparsity in [0.001]:
    for mode in ["left", "both"]:
        train_model_random(tcga_pathway, n_hidden=5500, num_epoch=1000, mode=mode, sparsity=sparsity,
            extra="5500_hidden_random_sparse_{1}_{0}".format(mode, sparsity))

epoch 0
training mse: 0.5927379 validation mse 0.604196
epoch time: 5.288846015930176
epoch 10
training mse: 0.20086 validation mse 0.22592442
epoch time: 2.948474407196045
epoch 20
training mse: 0.15078259 validation mse 0.18808544
epoch time: 2.9601621627807617
epoch 30
training mse: 0.12178384 validation mse 0.17031631
epoch time: 3.0083060264587402
epoch 40
training mse: 0.10132204 validation mse 0.15952392
epoch time: 2.992415428161621
epoch 50
training mse: 0.08616658 validation mse 0.15242074
epoch time: 2.995379686355591
epoch 60
training mse: 0.074940056 validation mse 0.1475617
epoch time: 2.9606404304504395
epoch 70
training mse: 0.06617726 validation mse 0.14376137
epoch time: 2.972929000854492
epoch 80
training mse: 0.059859656 validation mse 0.1413004
epoch time: 2.9888193607330322
epoch 90
training mse: 0.05460671 validation mse 0.13909756
epoch time: 2.9606916904449463
epoch 100
training mse: 0.05036956 validation mse 0.13727604
epoch time: 2.9577324390411377
epoch 110


In [52]:
def TF_layer(A_prev, name="bio-sparse"):
    mask = pd.read_hdf("../data/TF_network/TF_gene_matrix_filtered_by_tcga.hdf", "mask")
    size_in, size_out = mask.shape
    mask = tf.constant(mask.as_matrix().astype("float32"))
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        w_masked = tf.multiply(mask, w)
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w_masked) + b
        return act, w, w_masked, mask, b 
        

def build_model_TF_network(x, mode="left"):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = TF_layer(x, name="bio-sparse")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    
    if mode=="left":
        size_out, size_in = int(mask1.shape[0]), int(mask1.shape[1])
        x_recon, w2, b2 = fc_layer(hidden, size_in, size_out, name="fc1")
        parameters.update({"w2": w2, "b2": b2})
    elif mode == "both":
        mask2 =tf.transpose(mask1)
        size_in, size_out = int(mask2.shape[0]), int(mask2.shape[1])
        w2_init = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        w2_sparse = tf.multiply(mask2, w2_init)
        b2 = tf.Variable(tf.constant(0.1, shape=[size_out]))
        x_recon = tf.matmul(hidden, w2_sparse) + b2
        parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
            "mask2": mask2, "b2": b2})     
    else:
        raise
    return x_recon, parameters


def feed_forward_TF_network(x, parameters, mode="left"):
    w1_sparse, b1 = parameters["w1_sparse"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1_sparse) + b1)
    if mode == "left":
        w2, b2 = parameters["w2"], parameters["b2"]
    elif mode == "both":
        w2, b2 = parameters["w2_sparse"] ,parameters["b2"]
    else:
        raise
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden

In [47]:
def train_model_TF_network(data, batch_size=128, num_epoch=1000, 
                           learning_rate=1e-3, mode="left", extra=""):
    tf.reset_default_graph()
    LOGDIR = "/tmp/tcga_{0}".format(datetime.datetime.today().date())
    
    # train step
    (train_batch, train_iter, val_all, val_iter, 
        train_all, train_iter_all) = data.prep_batch(batch_size=batch_size, 
                                                     count_by="epoch")
    x = train_batch["X"]
    x_recon, parameters = build_model_TF_network(x, mode=mode)
    train_step = back_prop(x, x_recon, learning_rate)    
    
    # mse
    x_train, x_val = train_all["X"], val_all["X"]
    x_train_recon, train_latent = feed_forward_TF_network(x_train, parameters, mode=mode)
    x_val_recon, val_latent = feed_forward_TF_network(x_val, parameters, mode=mode)
    train_mse, train_summ = mse(x_train, x_train_recon, name="train")
    val_mse, val_summ = mse(x_val, x_val_recon, name="valiation")
    
    # run
    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + "ae_{0}".format(extra))
    writer.add_graph(sess.graph)                            
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epoch):
        sess.run([train_iter.initializer])
        t0 = time.time()
        try:
            while True:
                sess.run(train_step)
        except tf.errors.OutOfRangeError:
            sess.run([train_iter_all.initializer, val_iter.initializer])
            [train_error, train_s, val_error, val_s] = sess.run(
                [train_mse, train_summ, val_mse, val_summ])
            writer.add_summary(train_s, epoch)
            writer.add_summary(val_s, epoch)
            if epoch % 10 == 0:
                print("epoch", epoch)
                print("training mse:", train_error, "validation mse", val_error)
                print("epoch time:", time.time()-t0)  
    sess.close()

In [8]:
tcga_pathway = load_data.read_data_sets(
    "../data/TF_network/mRNA_lognorm_StandardScaled_pathway_genes.hdf")

In [53]:
for mode in ["both"]:
    train_model_TF_network(tcga_pathway, num_epoch=1000, mode=mode, 
                           extra="TF_network_sparse_{0}".format(mode))

epoch 0
training mse: 0.93593585 validation mse 0.9264781
epoch time: 11.377406358718872
epoch 10
training mse: 0.4717485 validation mse 0.4673784
epoch time: 3.204188823699951
epoch 20
training mse: 0.41342133 validation mse 0.40974596
epoch time: 3.2118191719055176
epoch 30
training mse: 0.39847544 validation mse 0.39497766
epoch time: 3.191128730773926
epoch 40
training mse: 0.3896634 validation mse 0.38624182
epoch time: 3.209786891937256
epoch 50
training mse: 0.38418385 validation mse 0.3807922
epoch time: 3.2109735012054443
epoch 60
training mse: 0.3805112 validation mse 0.37713966
epoch time: 3.213521718978882
epoch 70
training mse: 0.3779678 validation mse 0.374563
epoch time: 3.2344577312469482
epoch 80
training mse: 0.37617072 validation mse 0.37273845
epoch time: 3.227200984954834
epoch 90
training mse: 0.37500533 validation mse 0.37157983
epoch time: 3.2343859672546387
epoch 100
training mse: 0.37405384 validation mse 0.37064955
epoch time: 3.200198173522949
epoch 110
trai

epoch 910
training mse: 0.36841357 validation mse 0.36502257
epoch time: 3.218006134033203
epoch 920
training mse: 0.3683674 validation mse 0.3649963
epoch time: 3.2376067638397217
epoch 930
training mse: 0.36840606 validation mse 0.36504743
epoch time: 3.233531951904297
epoch 940
training mse: 0.36835873 validation mse 0.36502698
epoch time: 3.211911201477051
epoch 950
training mse: 0.36838007 validation mse 0.3650395
epoch time: 3.2057337760925293
epoch 960
training mse: 0.36835527 validation mse 0.36500624
epoch time: 3.228374719619751
epoch 970
training mse: 0.36837825 validation mse 0.3649882
epoch time: 3.2440805435180664
epoch 980
training mse: 0.36835417 validation mse 0.364966
epoch time: 3.2063512802124023
epoch 990
training mse: 0.3683502 validation mse 0.36500385
epoch time: 3.2377266883850098
